In [1]:
import numpy as np
import tritonclient.http

In [2]:
url = "10.10.66.25:8000"
model_name = "speech-video-handler"
model_version = "1"
batch_size = 1

In [3]:
triton_client = tritonclient.http.InferenceServerClient(url=url, verbose=False)

In [4]:
triton_client.is_model_ready(model_name=model_name, model_version=model_version)

True

In [5]:
model_metadata = triton_client.get_model_metadata(model_name=model_name, model_version=model_version)

In [6]:
model_config = triton_client.get_model_config(model_name=model_name, model_version=model_version)

### Video handler requests

In [7]:
_input = tritonclient.http.InferInput(name="video_url", shape=(batch_size,), datatype="BYTES")
output_1 = tritonclient.http.InferRequestedOutput(name="AUDIO_TEXT", binary_data=False)
# output_2 = tritonclient.http.InferRequestedOutput(name="LANGUAGE", binary_data=False)

In [8]:
video_url = "https://cdn-st.rutubelist.ru/media/d1/e7/642dc2194fcdb69664f832d5f2dd/fhd.mp4"

In [9]:
_input.set_data_from_numpy(np.asarray([video_url] * batch_size, dtype=object))

response = triton_client.infer(
        model_name=model_name,
        model_version=model_version,
        inputs=[_input],
        outputs=[output_1],
    )

print(response.get_response())

{'model_name': 'speech-video-handler', 'model_version': '1', 'outputs': [{'name': 'AUDIO_TEXT', 'datatype': 'BYTES', 'shape': [], 'data': ['я убежден что чем беднее человек тем сложнее его удовлетворить потому что тот человек у которого нет денег он всегда недоволен чем он всегда просит больше который в принципе удовлетворен своими деньгами он говорит классно это здорово спасибо и идет и делает']}]}


### Xclip handler requests

In [11]:
model_name = "xclip-video-handler"

_input = tritonclient.http.InferInput(name="video_url", shape=(batch_size,), datatype="BYTES")
output_1 = tritonclient.http.InferRequestedOutput(name="EMBEDDING", binary_data=False)

In [12]:
video_url = "https://cdn-st.rutubelist.ru/media/d1/e7/642dc2194fcdb69664f832d5f2dd/fhd.mp4"
_input.set_data_from_numpy(np.asarray([video_url] * batch_size, dtype=object))

response = triton_client.infer(
        model_name=model_name,
        model_version=model_version,
        inputs=[_input],
        outputs=[output_1],
    )

print(response.get_response())

InferenceServerException: [500] Failed to process the request(s) for model instance 'xclip-video-handler_0_0', message: BdbQuit: <EMPTY MESSAGE>

At:
  /usr/lib/python3.10/bdb.py(115): dispatch_line
  /usr/lib/python3.10/bdb.py(90): trace_dispatch
  /src/models/xclip-video-handler/1/model.py(67): execute


In [ ]:
triton_client.close()